## 선형 회귀(Linear Regression) <br>
오차가 가장 작은 직선을 구하는 문제<br>
생선의 길이로 무게를 예측하는 문제

In [ ]:
import numpy as np                                     # 넘파이 패키지
import pandas as pd                                    # 판다스 패키지
import matplotlib.pyplot as plt                        # 그래프 패키지
from sklearn.model_selection import train_test_split   # 데이터 셋 분할

from sklearn.linear_model import LinearRegression      # 선형 회귀 패키지
from sklearn.linear_model import Ridge                 # 릿지 선형 회귀 패키지
from sklearn.preprocessing import PolynomialFeatures   # 특성 변환기 패키지

from keras.models import Sequential
from keras import optimizers
import tensorflow as tf

from keras.callbacks import EarlyStopping
from keras.models import load_model

from sklearn.metrics import mean_absolute_error        # 평균 절대값 에러(모델 평가 지수)
from sklearn.metrics import mean_squared_error         # 평균 제곱근 에러(모델 평가 지수)

In [ ]:
# 코랩과 구굴 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RNN/data/mokpo_data.csv')
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30624 entries, 0 to 30623
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기온      30624 non-null  float64
 1   풍속      30624 non-null  float64
 2   습도      30624 non-null  int64  
 3   기압      30624 non-null  float64
 4   일조      30624 non-null  float64
 5   일사      30624 non-null  float64
 6   전운량     30624 non-null  int64  
 7   발전량     30624 non-null  float64
dtypes: float64(6), int64(2)
memory usage: 1.9 MB
None
    기온   풍속  습도      기압   일조  일사   전운량  발전량
0 -3.6  1.9  66  1027.7  0.0  0.0    9  0.0
1 -4.4  2.3  66  1027.6  0.0  0.0    6  0.0
2 -4.1  2.1  65  1028.1  0.0  0.0    9  0.0
3 -4.0  1.1  63  1028.0  0.0  0.0    9  0.0
4 -4.2  1.3  63  1027.5  0.0  0.0    9  0.0


In [ ]:
data = np.array(data)          # 데이터 프레임을 배열로 변환

data = (data - np.min(data, 0)) / (np.max(data, 0) - np.min(data, 0))
print(data.shape)              # 데이터 스케일 보기
print(len(data) * 0.8)         # 데이터 나누기 위해서

(30624, 8)
24499.2


In [ ]:
x_data = data[ : -1, : ]             # 입력 데이터(마지막 샘플 전까지)
y_data = data[1 : , -1]              # 타겟 데이터(다음날 부터 마지막 샘플까지)
print(x_data.shape)
print(y_data.shape)
print(y_data[:5])

(30623, 8)
(30623,)
[0. 0. 0. 0. 0.]


In [ ]:
# 데이터 셋 만들기
x_train = x_data[ : 24499 ,  : ]         # 학습 데이터
y_train = y_data[ : 24499]

x_test = x_data[24499 : ,  : ]          # 테스트 데이터
y_test = y_data[24499 :]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(24499, 8)
(24499,)
(6124, 8)
(6124,)


<h3>다중회귀(multiple regression)</h3>
*   여러 개의 특성을 이용한 선형회귀<br>
*   선형 회귀 방정식<br>
*   타깃 = a × (특성1) + b × (특성2) + ..... + 절편

In [ ]:
# 학습할 수 있는 모델을 만듬
lr = LinearRegression()                     # 선형 회귀 모델
lr.fit(x_train, y_train)                    # 모델 훈련

print(lr.score(x_train, y_train))           # 훈련 셋 평가
print(lr.score(x_test, y_test))             # 테스트 셋 평가

0.8794741049746468
0.8716889678746736


In [ ]:
# 테스트 세트에 대한 예측 값
test_prediction = lr.predict(x_test)

# 테스트 세트에 대한 평균 절댓값 오차
mae = mean_absolute_error(y_test, test_prediction)
mse = mean_squared_error(y_test, test_prediction)
rmse = mse**0.5
print(mae)
print(mse)
print(rmse)

0.05590270487128472
0.008161603057721883
0.0903415909629772


##특성 공학
기존 특성을 이용해 새로운 특성을 생성<br>
degree=2

In [ ]:
poly = PolynomialFeatures(include_bias=False)     # 특성 변환기 객체(절편 없앰)
poly.fit(x_train)                                 # 8개의 특성을 이용하여 특성을 늘리는 학습

train_poly = poly.transform(x_train)              # 여기서 훈련 셋 특성 변함
test_poly = poly.transform(x_test)                # 테스트 셋 특성 변환

poly.get_feature_names_out()                      # 변환 특성 확인
print(train_poly.shape)                           # 특성 변환 스케알

(24499, 44)


In [ ]:
lr = LinearRegression()                          # 선형 회귀 모델
lr.fit(train_poly, y_train)                      # 모델 훈련

print(lr.score(train_poly, y_train))             # 훈련 셋 평가
print(lr.score(test_poly, y_test))               # 테스트 셋 평가

0.8969992717449276
0.8927687236177932


In [ ]:
test_prediction = lr.predict(test_poly)

mae = mean_absolute_error(y_test, test_prediction)
mse = mean_squared_error(y_test, test_prediction)
rmse = mse**0.5
print(mae)
print(mse)
print(rmse)

0.04777431921468522
0.006820762787954337
0.0825879094538319


degree=4

In [ ]:
poly = PolynomialFeatures(degree=4, include_bias=False)
poly.fit(x_train)

train_poly = poly.transform(x_train)
test_poly = poly.transform(x_test)

poly.get_feature_names_out()
print(train_poly.shape)

(24499, 494)


In [ ]:
lr = LinearRegression()
lr.fit(train_poly, y_train)
print(lr.score(train_poly, y_train))
print(lr.score(test_poly, y_test))

0.9154697941620656
0.9012717124327756


In [ ]:
# 테스트 세트에 대한 예측 값
test_prediction = lr.predict(test_poly)

mae = mean_absolute_error(y_test, test_prediction)
mse = mean_squared_error(y_test, test_prediction)
rmse = mse**0.5
print(mae)
print(mse)
print(rmse)

0.044103830193033586
0.006279905011638175
0.07924585169987244


degree=5

In [ ]:
poly = PolynomialFeatures(degree=5, include_bias=False)
poly.fit(x_train)

train_poly = poly.transform(x_train)
test_poly = poly.transform(x_test)

poly.get_feature_names_out()
print(train_poly.shape)

(24499, 1286)


In [ ]:
lr = LinearRegression()
lr.fit(train_poly, y_train)
print(lr.score(train_poly, y_train))
print(lr.score(test_poly, y_test))

0.9247809189180961
0.8586134747887589


In [ ]:
# 테스트 세트에 대한 예측 값
test_prediction = lr.predict(test_poly)
# 테스트 세트에 대한 평균 절댓값 오차를 계산합니다
mae = mean_absolute_error(y_test, test_prediction)
mse = mean_squared_error(y_test, test_prediction)
rmse = mse**0.5
print(mae)
print(mse)
print(rmse)

0.04646922291971118
0.008993308504896436
0.09483305597151467


##규제

In [ ]:
ridge = Ridge()                             # 릿지 선형 회귀 모델
ridge.fit(train_poly, y_train)              # 훈련

print(ridge.score(train_poly, y_train))     # 훈련 셋 평가
print(ridge.score(test_poly, y_test))       # 테스트 셋 평가

0.9076175478929601
0.9039611136363551


In [ ]:
test_prediction = ridge.predict(test_poly)

mae = mean_absolute_error(y_test, test_prediction)
mse = mean_squared_error(y_test, test_prediction)
rmse = mse**0.5
print(mae)
print(mse)
print(rmse)

0.04361946256004331
0.006108837686225842
0.07815905377002617
